# Alpaca-py stock trading basic

- This notebook shows how to use alpaca-py with stock trading API endpoints
- Please use ``paper account``. Please ``DO NOT`` use this notebook with live account. In this notebook, we place orders for stocks as an example.

In [40]:
# Please change the following to your own PAPER api key and secret
# or set them as environment variables (ALPACA_API_KEY, ALPACA_SECRET_KEY).
# You can get them from https://alpaca.markets/

api_key = None
secret_key = None

#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
stream_data_wss = None

In [41]:
import os

# Check if environment variables are set
if "ALPACA_API_KEY" in os.environ:
    api_key = os.getenv("ALPACA_API_KEY")
if "ALPACA_SECRET_KEY" in os.environ:
    secret_key = os.getenv("ALPACA_SECRET_KEY")
if "ALPACA_BASE_URL" in os.environ:
    trade_api_url = os.getenv("ALPACA_BASE_URL")



In [42]:
# install alpaca-py if it is not available
try:
    import alpaca
except ImportError:
    !python3 -m pip install alpaca-py
    import alpaca

In [43]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.corporate_actions import CorporateActionsClient
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.stock import StockDataStream

from alpaca.data.requests import (
    CorporateActionsRequest,
    StockBarsRequest,
    StockQuotesRequest,
    StockTradesRequest,
)
from alpaca.trading.requests import (
    ClosePositionRequest,
    GetAssetsRequest,
    GetOrdersRequest,
    LimitOrderRequest,
    MarketOrderRequest,
    StopLimitOrderRequest,
    StopLossRequest,
    StopOrderRequest,
    TakeProfitRequest,
    TrailingStopOrderRequest,
)
from alpaca.trading.enums import (
    AssetExchange,
    AssetStatus,
    OrderClass,
    OrderSide,
    OrderType,
    QueryOrderStatus,
    TimeInForce,
)

In [44]:
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [45]:
# check version of alpaca-py
alpaca.__version__

'0.40.1'

# Trading Client

In [46]:
# setup clients
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper, url_override=trade_api_url)

In [47]:
# check trading account
# You can check definition of each field in the following documents
# ref. https://docs.alpaca.markets/docs/account-plans
# ref. https://docs.alpaca.markets/reference/getaccount-1
acct = trade_client.get_account()
acct

{   'account_blocked': False,
    'account_number': 'PA3GPJJBCV6D',
    'accrued_fees': '0',
    'buying_power': '180529.48',
    'cash': '80504.39',
    'created_at': datetime.datetime(2025, 6, 9, 15, 56, 50, 893330, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 3,
    'daytrading_buying_power': '0',
    'equity': '100025.09',
    'id': UUID('8273af54-8bd2-40e0-be96-857620779ffe'),
    'initial_margin': '9760.35',
    'last_equity': '99999.19622353472',
    'last_maintenance_margin': '3775.05',
    'long_market_value': '19520.7',
    'maintenance_margin': '5856.21',
    'multiplier': '2',
    'non_marginable_buying_power': '88264.73',
    'options_approved_level': 3,
    'options_buying_power': '88264.73',
    'options_trading_level': 3,
    'pattern_day_trader': False,
    'pending_transfer_in': None,
    'pending_transfer_out': None,
    'portfolio_value': '100025.09',
    'regt_buying_power': '180529.48',
  

In [48]:
# check account configuration
# ref. https://docs.alpaca.markets/reference/getaccountconfig-1
acct_config = trade_client.get_account_configurations()
acct_config

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': True,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

In [49]:
# set account configuration
# ref. https://docs.alpaca.markets/reference/patchaccountconfig-1
req = acct_config
req.fractional_trading = not req.fractional_trading # toggle fractional trading
acct_config_new = trade_client.set_account_configurations(req)
display(acct_config_new)

# revert changes
req = acct_config_new
req.fractional_trading = not req.fractional_trading # toggle fractional trading
acct_config_reverted = trade_client.set_account_configurations(req)
display(acct_config_reverted)

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': False,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': True,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

In [50]:
# get list of assets which are us_equity (default), active, and in NASDAQ
# ref. https://docs.alpaca.markets/reference/get-v2-assets-1
req = GetAssetsRequest(
  # asset_class=AssetClass.US_EQUITY,  # default asset_class is us_equity
  status=AssetStatus.ACTIVE,
  exchange=AssetExchange.NASDAQ,
)
assets = trade_client.get_all_assets(req)
print(f"Number of assets: {len(assets)}")
assets[:2]

Number of assets: 4950


[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': [],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.NASDAQ: 'NASDAQ'>,
     'fractionable': False,
     'id': UUID('7af0066a-e334-4de7-8222-9a75376a5766'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
     'name': 'E-Home Household Service Holdings Limited Ordinary Shares',
     'price_increment': None,
     'shortable': False,
     'status': <AssetStatus.ACTIVE: 'active'>,
     'symbol': 'EJH',
     'tradable': False},
 {   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': ['has_options'],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.NASDAQ: 'NASDAQ'>,
     'fractionable': False,
     'id': UUID('cc492b87-aff1-442a-99bb-e4e638dd6e98'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
  

### Orders

In [51]:
# we will place orders which Alapca trading platform supports
# - order classes: simple, bracket, oco, oto
# - order types: market, limit, stop, stop_limit, trailing_stop
#
# please refer to the following documents for more details
# ref. https://docs.alpaca.markets/docs/orders-at-alpaca
# ref. https://docs.alpaca.markets/reference/postorder
#
# we will also use fractional trading capability of Alpaca trading platform in this example
# ref. https://docs.alpaca.markets/docs/fractional-trading

# we will place orders for symbol: SPY in this example
symbol = "SPY"

In [52]:
# simple, market order, fractional qty
# Alpaca trading platform support fractional trading by default
# you can specify:
# fractional qty (e.g. 0.01 qty) in the order request (which is shown in this example)
# or notional value (e.g. 100 USD) (which is in the next example)
#
# If you have an error of `qty must be integer`,
# please try to `Reset Account` of your paper account via the Alpaca Trading API dashboard
req = MarketOrderRequest(
    symbol = symbol,
    qty = 5.5, # Example: place an order for $100 dollars worth of SPY
    #notional = 100, # Example: place an order for 5.5 shares of SPY
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': 'e980cbe8-007b-4a88-9b03-b3165912c510',
    'created_at': datetime.datetime(2025, 6, 10, 15, 0, 12, 107271, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 6, 10, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('443f4b7c-a781-4aa3-ad80-d967a553bad9'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '5.5',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderSta

In [53]:
# simple, market order, notional
# Alpaca trading platform support fractional trading by default
# you can specify:
# fractional qty (e.g. 0.01 qty) in the order request (which is in the example above)
# or notional value (e.g. 100 USD) (which is in this example)
req = MarketOrderRequest(
    symbol = symbol,
    notional = 1.11,  # notional is specified in USD, here we specify $1.11
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': '1c8af5dd-f47c-4fdb-aebe-5cc1b11a8290',
    'created_at': datetime.datetime(2025, 6, 10, 15, 0, 15, 39294, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 6, 10, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('9281f136-7f33-40e9-aac6-fcdd9e2f963e'),
    'legs': None,
    'limit_price': None,
    'notional': '1.11',
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': None,
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderSta

In [54]:
# simple, limit order, fractional qty
req = LimitOrderRequest(
    symbol = symbol,
    qty = 0.01,
    limit_price = 550.25,
    side = OrderSide.BUY,
    type = OrderType.LIMIT,
    time_in_force = TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': '69b61f52-d908-4319-9a73-bb7821d9a5d0',
    'created_at': datetime.datetime(2025, 6, 10, 15, 0, 17, 834538, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 6, 10, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('8966b2a5-cb50-4692-8ee8-0addf55ce46f'),
    'legs': None,
    'limit_price': '550.25',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.LIMIT: 'limit'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '0.01',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <Order

In [55]:
# stop order
req = StopOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.GTC,
                    stop_price = 600
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': 'da1ce005-2ae0-4f52-a4b7-a04eb8654119',
    'created_at': datetime.datetime(2025, 6, 10, 15, 0, 20, 534381, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 9, 8, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('aa7c1e11-e753-4f46-a45c-93dd2844717d'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.STOP: 'stop'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '1',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.PEN

In [56]:
# stop limit order
req = StopLimitOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.GTC,
                    limit_price = 550,
                    stop_price = 600
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': '1efb8174-5862-4903-9cee-8ac7a8708729',
    'created_at': datetime.datetime(2025, 6, 10, 15, 0, 22, 813560, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 9, 8, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('3b863328-8c73-46fe-8311-479e5d107387'),
    'legs': None,
    'limit_price': '550',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.STOP_LIMIT: 'stop_limit'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '1',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <Or

In [57]:
# bracket order with both stop loss and take profit
req = MarketOrderRequest(
                    symbol = symbol,
                    qty = 5,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.DAY,
                    order_class = OrderClass.BRACKET,
                    take_profit = TakeProfitRequest(limit_price=602),  # Adjusted: e.g., 602 or higher based on base_price
                    stop_loss = StopLossRequest(stop_price=300)
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': '408a6dfd-1531-4526-b04c-cbbee1be6c15',
    'created_at': datetime.datetime(2025, 6, 10, 15, 0, 24, 711331, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 6, 10, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('3d92cc24-0511-410c-b7b9-011bfb5e120f'),
    'legs': [   {   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
                    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
                    'canceled_at': None,
                    'client_order_id': 'ac0c638c-0a92-4fd3-934d-db1c3a101179',
                    'created_at': datetime.datetime(2025, 6, 10, 15, 0, 24, 711331, tzinfo=TzInfo(UTC)),
                    'expired

In [58]:
# oto order with stop loss
req = LimitOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    limit_price = 500,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.DAY,
                    Class = OrderClass.OTO,
                    stop_loss = StopLossRequest(stop_price = 300)
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': 'ec51f65b-7da7-46aa-9d5e-7860c0e18e18',
    'created_at': datetime.datetime(2025, 6, 10, 15, 0, 27, 177764, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 6, 10, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('509a19c3-8cd3-4fb4-932c-6c6a858f3c04'),
    'legs': None,
    'limit_price': '500',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.LIMIT: 'limit'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '1',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus

In [59]:
# oco limit order
req = LimitOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    limit_price = 500,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.DAY,
                    Class = OrderClass.OCO
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': 'f814eb89-9238-46df-b007-959b540c92da',
    'created_at': datetime.datetime(2025, 6, 10, 15, 0, 29, 542762, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 6, 10, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('b37453c3-e0e4-4ed3-bd17-799646f3a390'),
    'legs': None,
    'limit_price': '500',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.LIMIT: 'limit'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '1',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus

In [60]:
# trailing stop order
req = TrailingStopOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    side = OrderSide.SELL,
                    time_in_force = TimeInForce.GTC,
                    trail_percent = 0.20 # you can also use trail_price instead of trail_percent
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': '333af33e-b9b4-4197-bf9c-61a3517c6d08',
    'created_at': datetime.datetime(2025, 6, 10, 15, 0, 31, 975282, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 9, 8, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': '600.96',
    'id': UUID('a04a4c27-0e0b-4481-8362-912ff63caff2'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.TRAILING_STOP: 'trailing_stop'>,
    'position_intent': <PositionIntent.SELL_TO_CLOSE: 'sell_to_close'>,
    'qty': '1',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.SELL: 'sell'>,
  

In [61]:
# get a list of orders including closed (e.g. filled) orders by specifying symbol
req = GetOrdersRequest(
    status = QueryOrderStatus.ALL,
    symbols = [symbol]
)
orders = trade_client.get_orders(req)
orders

[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
     'canceled_at': None,
     'client_order_id': '333af33e-b9b4-4197-bf9c-61a3517c6d08',
     'created_at': datetime.datetime(2025, 6, 10, 15, 0, 31, 975282, tzinfo=TzInfo(UTC)),
     'expired_at': None,
     'expires_at': datetime.datetime(2025, 9, 8, 20, 0, tzinfo=TzInfo(UTC)),
     'extended_hours': False,
     'failed_at': None,
     'filled_at': None,
     'filled_avg_price': None,
     'filled_qty': '0',
     'hwm': '600.97',
     'id': UUID('a04a4c27-0e0b-4481-8362-912ff63caff2'),
     'legs': None,
     'limit_price': None,
     'notional': None,
     'order_class': <OrderClass.SIMPLE: 'simple'>,
     'order_type': <OrderType.TRAILING_STOP: 'trailing_stop'>,
     'position_intent': <PositionIntent.SELL_TO_CLOSE: 'sell_to_close'>,
     'qty': '1',
     'ratio_qty': None,
     'replaced_at': None,
     'replaced_by': None,
     'replaces': None,
     'side': <O

In [62]:
# see all open orders
req = GetOrdersRequest(
    status = QueryOrderStatus.OPEN,
    symbols = [symbol]
)
open_orders = trade_client.get_orders(req)
open_orders

[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
     'canceled_at': None,
     'client_order_id': '333af33e-b9b4-4197-bf9c-61a3517c6d08',
     'created_at': datetime.datetime(2025, 6, 10, 15, 0, 31, 975282, tzinfo=TzInfo(UTC)),
     'expired_at': None,
     'expires_at': datetime.datetime(2025, 9, 8, 20, 0, tzinfo=TzInfo(UTC)),
     'extended_hours': False,
     'failed_at': None,
     'filled_at': None,
     'filled_avg_price': None,
     'filled_qty': '0',
     'hwm': '600.97',
     'id': UUID('a04a4c27-0e0b-4481-8362-912ff63caff2'),
     'legs': None,
     'limit_price': None,
     'notional': None,
     'order_class': <OrderClass.SIMPLE: 'simple'>,
     'order_type': <OrderType.TRAILING_STOP: 'trailing_stop'>,
     'position_intent': <PositionIntent.SELL_TO_CLOSE: 'sell_to_close'>,
     'qty': '1',
     'ratio_qty': None,
     'replaced_at': None,
     'replaced_by': None,
     'replaces': None,
     'side': <O

In [63]:
# cancel all open orders
trade_client.cancel_orders()

[{   'body': None,
     'id': UUID('3b863328-8c73-46fe-8311-479e5d107387'),
     'status': 200},
 {   'body': None,
     'id': UUID('509a19c3-8cd3-4fb4-932c-6c6a858f3c04'),
     'status': 200},
 {   'body': None,
     'id': UUID('703b98ce-2045-48d1-bc61-081fa0e69457'),
     'status': 200},
 {   'body': None,
     'id': UUID('8966b2a5-cb50-4692-8ee8-0addf55ce46f'),
     'status': 200},
 {   'body': None,
     'id': UUID('a04a4c27-0e0b-4481-8362-912ff63caff2'),
     'status': 200},
 {   'body': None,
     'id': UUID('b37453c3-e0e4-4ed3-bd17-799646f3a390'),
     'status': 200}]

### Positions

In [64]:
# get all open positions
# ref. https://docs.alpaca.markets/reference/getallopenpositions-1
positions = trade_client.get_all_positions()
positions

[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
     'asset_marginable': True,
     'avg_entry_price': '600.497325',
     'avg_entry_swap_rate': None,
     'change_today': '0.0022345250800427',
     'cost_basis': '26408.267501',
     'current_price': '601.02',
     'exchange': <AssetExchange.ARCA: 'ARCA'>,
     'lastday_price': '599.68',
     'market_value': '26431.253342',
     'qty': '43.977327446',
     'qty_available': '43.977327446',
     'side': <PositionSide.LONG: 'long'>,
     'swap_rate': None,
     'symbol': 'SPY',
     'unrealized_intraday_pl': '58.929619',
     'unrealized_intraday_plpc': '0.0022345250884588',
     'unrealized_pl': '22.985841',
     'unrealized_plpc': '0.0008704032174443',
     'usd': None}]

In [65]:
# get positions by symbol
# ref. https://docs.alpaca.markets/reference/getopenposition-1
position = trade_client.get_open_position(symbol_or_asset_id=symbol)
position


{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'asset_marginable': True,
    'avg_entry_price': '600.497325',
    'avg_entry_swap_rate': None,
    'change_today': '0.0022595384204909',
    'cost_basis': '26408.267501',
    'current_price': '601.035',
    'exchange': <AssetExchange.ARCA: 'ARCA'>,
    'lastday_price': '599.68',
    'market_value': '26431.913002',
    'qty': '43.977327446',
    'qty_available': '43.977327446',
    'side': <PositionSide.LONG: 'long'>,
    'swap_rate': None,
    'symbol': 'SPY',
    'unrealized_intraday_pl': '59.589279',
    'unrealized_intraday_plpc': '0.0022595384322554',
    'unrealized_pl': '23.645501',
    'unrealized_plpc': '0.0008953825160664',
    'usd': None}

In [66]:
# get positions by asset_id
trade_client.get_open_position(symbol_or_asset_id=position.asset_id)

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'asset_marginable': True,
    'avg_entry_price': '600.497325',
    'avg_entry_swap_rate': None,
    'change_today': '0.0022345250800427',
    'cost_basis': '26408.267501',
    'current_price': '601.02',
    'exchange': <AssetExchange.ARCA: 'ARCA'>,
    'lastday_price': '599.68',
    'market_value': '26431.253342',
    'qty': '43.977327446',
    'qty_available': '43.977327446',
    'side': <PositionSide.LONG: 'long'>,
    'swap_rate': None,
    'symbol': 'SPY',
    'unrealized_intraday_pl': '58.929619',
    'unrealized_intraday_plpc': '0.0022345250884588',
    'unrealized_pl': '22.985841',
    'unrealized_plpc': '0.0008704032174443',
    'usd': None}

In [67]:
# close the position with specifying qty
# ref. https://docs.alpaca.markets/reference/deleteopenposition-1
trade_client.close_position(
    symbol_or_asset_id = symbol,
    close_options = ClosePositionRequest(
        qty = "0.01",
    )
)

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': 'ccc83c38-913b-4b6c-a6f2-d685dacc7d71',
    'created_at': datetime.datetime(2025, 6, 10, 15, 0, 51, 861963, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 6, 10, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('183316bf-8d8d-4717-8c2c-123664fa754e'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'position_intent': <PositionIntent.SELL_TO_CLOSE: 'sell_to_close'>,
    'qty': '0.01',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.SELL: 'sell'>,
    'status': <O

# Trade Update (Stream)

With TradingStream client, you can get updates about trades

You can open this notebook in another window and run below cell to check trade updates.

In [ ]:
# subscribe trade updates
trade_stream_client = TradingStream(api_key, secret_key, paper=paper, url_override = trade_api_wss)

async def trade_updates_handler(data):
    print(data)

trade_stream_client.subscribe_trade_updates(trade_updates_handler)
trade_stream_client.run()

# Market Data (Historical)

In [68]:
# setup stock historical data client
stock_historical_data_client = StockHistoricalDataClient(api_key, secret_key, url_override = data_api_url)

In [69]:
# get historical bars by symbol
# ref. https://docs.alpaca.markets/reference/stockbars-1
now = datetime.now(ZoneInfo("America/New_York"))
req = StockBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe=TimeFrame(amount = 1, unit = TimeFrameUnit.Hour), # specify timeframe
    start = now - timedelta(days = 5),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    limit = 2,                                               # specify limit
)
stock_historical_data_client.get_stock_bars(req).df

open      high      low   close  \
symbol timestamp                                                      
SPY    2025-06-05 16:00:00+00:00  598.76  598.9500  594.260  594.76   
       2025-06-05 17:00:00+00:00  594.75  596.9797  594.305  595.82   

                                     volume  trade_count        vwap  
symbol timestamp                                                      
SPY    2025-06-05 16:00:00+00:00  9659760.0     113349.0  596.676118  
       2025-06-05 17:00:00+00:00  6908398.0      82166.0  595.978784

In [70]:
# get historical trades by symbol
req = StockTradesRequest(
    symbol_or_symbols = [symbol],
    start = now - timedelta(days = 5),                          # specify start datetime, default=the beginning of the current day.
    # end=None,                                             # specify end datetime, default=now
    limit = 2,                                                # specify limit
)
stock_historical_data_client.get_stock_trades(req).df

exchange   price  size  \
symbol timestamp                                                 
SPY    2025-06-05 15:00:56.846242+00:00        V  597.38  80.0   
       2025-06-05 15:00:56.846801+00:00        J  597.38   3.0   

                                                     id conditions tape  
symbol timestamp                                                         
SPY    2025-06-05 15:00:56.846242+00:00  52983593189683     [ , I]    B  
       2025-06-05 15:00:56.846801+00:00  52983525225330     [ , I]    B

In [71]:
# get historical quotes by symbol
req = StockQuotesRequest(
    symbol_or_symbols = [symbol],
    start = now - timedelta(days = 5),                      # specify start datetime, default=the beginning of the current day.
    # end=None,                                             # specify end datetime, default=now
    limit = 2,                                              # specify limit
)
stock_historical_data_client.get_stock_quotes(req).df

bid_price  bid_size bid_exchange  \
symbol timestamp                                                            
SPY    2025-06-05 15:00:56.777639+00:00     597.37       7.0            P   
       2025-06-05 15:00:56.814642+00:00     597.37       9.0            P   

                                         ask_price  ask_size ask_exchange  \
symbol timestamp                                                            
SPY    2025-06-05 15:00:56.777639+00:00     597.38       3.0            T   
       2025-06-05 15:00:56.814642+00:00     597.38       3.0            T   

                                        conditions tape  
symbol timestamp                                         
SPY    2025-06-05 15:00:56.777639+00:00        [R]    B  
       2025-06-05 15:00:56.814642+00:00        [R]    B

In [72]:
# get latest quotes by symbol
req = StockQuotesRequest(
    symbol_or_symbols = [symbol],
)
res = stock_historical_data_client.get_stock_latest_quote(req)
res

{'SPY': {   'ask_exchange': 'V',
     'ask_price': 601.03,
     'ask_size': 1.0,
     'bid_exchange': 'V',
     'bid_price': 601.01,
     'bid_size': 1.0,
     'conditions': ['R'],
     'symbol': 'SPY',
     'tape': 'B',
     'timestamp': datetime.datetime(2025, 6, 10, 15, 1, 2, 617487, tzinfo=TzInfo(UTC))}}

# Market Data (Stream)

In [ ]:
stock_data_stream_client = StockDataStream(api_key, secret_key, url_override = stream_data_wss)

async def stock_data_stream_handler(data):
    print(data)

symbols = [symbol]

stock_data_stream_client.subscribe_quotes(stock_data_stream_handler, *symbols)
stock_data_stream_client.subscribe_trades(stock_data_stream_handler, *symbols)

stock_data_stream_client.run()

# Corporate actions

In [73]:
corporate_actions_client = CorporateActionsClient(api_key, secret_key)
corporate_actions_client.get_corporate_actions(CorporateActionsRequest(
    start=datetime(2020, 1, 1),
    symbols=[symbol]
)).df

,symbol,rate,special,foreign,process_date,ex_date,record_date,payable_date
corporate_action_type,,,,,,,,
cash_dividends,SPY,1.405559,False,False,2020-04-30,2020-03-20,2020-03-23,2020-04-30
cash_dividends,SPY,1.366242,False,False,2020-07-31,2020-06-19,2020-06-22,2020-07-31
cash_dividends,SPY,1.339224,False,False,2020-10-30,2020-09-18,2020-09-21,2020-10-30
cash_dividends,SPY,1.580000,False,False,2021-01-29,2020-12-18,2020-12-21,2021-01-29
cash_dividends,SPY,1.277788,False,False,2021-04-30,2021-03-19,2021-03-22,2021-04-30
cash_dividends,SPY,1.375875,False,False,2021-07-30,2021-06-18,2021-06-21,2021-07-30
cash_dividends,SPY,1.428117,False,False,2021-10-29,2021-09-17,2021-09-20,2021-10-29
cash_dividends,SPY,1.636431,False,False,2022-01-31,2021-12-17,2021-12-20,2022-01-31
cash_dividends,SPY,1.366009,False,False,2022-04-29,2022-03-18,2022-03-21,2022-04-29
